In [ ]:
# !pip install --upgrade transformers

In [ ]:
# !pip install auto-round

In [1]:
import os
import torch
from auto_round import AutoRound
from huggingface_hub import HfApi, create_repo

In [2]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")


PyTorch Version: 2.8.0+cu126
CUDA Available: True
CUDA Version: 12.6
GPU Name: NVIDIA L40
VRAM: 44.3 GB


In [4]:
MODEL_ID = "Qwen/Qwen3-VL-8B-Instruct"
HF_USER = "Vishva007"
OUTPUT_BASE_DIR = "./AutoRound"

In [5]:
HF_TOKEN = os.getenv("HF_TOKEN")

In [ ]:
TARGET_FORMATS = [
    "AWQ",            # Best for Nvidia GPUs (vLLM, TGI)
    "GPTQ",           # Good compatibility
    "AutoRound",      # Intel default format (Requires auto-round lib to run)
    # "GGUF",         # For llama.cpp / Olama
]

# High-End GPU Tuning Parameters (A40/A6000/L40)
TUNING_CONFIG = {
    "group_size": 128,
    "sym": True,
    "iters": 1000,          # High accuracy (Production grade)
    "nsamples": 512,        # More calibration data
    "batch_size": 8,        # Faster on 48GB VRAM
    "seqlen": 2048,
    "low_gpu_mem_usage": False,   # Keep on GPU for speed
    "enable_torch_compile": True, # JIT acceleration
    "quant_nontext_module": False # Keep Vision Tower in BF16 (Crucial for VLM accuracy)
}

In [12]:
def push_to_hub(local_dir, repo_name, token):
    """Creates repo and uploads folder to Hugging Face."""
    full_repo_id = f"{HF_USER}/{repo_name}"
    print(f"\n[Hub] Pushing {local_dir} to {full_repo_id}...")
    
    try:
        api = HfApi(token=token)
        create_repo(full_repo_id, repo_type="model", exist_ok=True, private=False, token=token)
        
        api.upload_folder(
            folder_path=local_dir,
            repo_id=full_repo_id,
            repo_type="model",
            token=token
        )
        print(f"[Hub] ✅ Successfully uploaded: https://huggingface.co/{full_repo_id}")
    except Exception as e:
        print(f"[Hub] ❌ Error uploading: {e}")

In [13]:
ar = AutoRound(
        MODEL_ID,
        scheme="W4A16",
        **TUNING_CONFIG
    )

2026-01-30 17:01:45 INFO autoround.py L158: using MLLM mode for multimodal model.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2026-01-30 17:01:53 INFO base.py L391: using torch.bfloat16 for quantization tuning


In [14]:
ar.quantize()

2026-01-30 17:02:05 WARNING utils.py L425: model.visual.blocks.0.mlp.linear_fc1 skipped quantization (shape not divisible by 32).
2026-01-30 17:02:05 WARNING utils.py L425: model.visual.blocks.0.mlp.linear_fc2 skipped quantization (shape not divisible by 32).
2026-01-30 17:02:05 WARNING utils.py L425: model.visual.blocks.1.mlp.linear_fc1 skipped quantization (shape not divisible by 32).
2026-01-30 17:02:05 WARNING utils.py L425: model.visual.blocks.1.mlp.linear_fc2 skipped quantization (shape not divisible by 32).
2026-01-30 17:02:05 WARNING utils.py L425: model.visual.blocks.2.mlp.linear_fc1 skipped quantization (shape not divisible by 32).
2026-01-30 17:02:05 WARNING utils.py L425: model.visual.blocks.2.mlp.linear_fc2 skipped quantization (shape not divisible by 32).
2026-01-30 17:02:05 WARNING utils.py L425: model.visual.blocks.3.mlp.linear_fc1 skipped quantization (shape not divisible by 32).
2026-01-30 17:02:05 WARNING utils.py L425: model.visual.blocks.3.mlp.linear_fc2 skipped qu

README.md:   0%|          | 0.00/373 [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/921 [00:00<?, ?B/s]

data/train-00000-of-00001-4746b8785c874c(…):   0%|          | 0.00/33.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1216 [00:00<?, ? examples/s]

cache block inputs: 100%|██████████| 512/512 [00:01<00:00, 272.40it/s]
2026-01-30 17:03:06 INFO base.py L1744: caching done
Quantizing model.language_model.layers.0:   0%|          | 0/36 [00:02<?, ?it/s]/opt/conda/lib/python3.11/site-packages/torch/autograd/graph.py:829: UserWarning: Flash Attention defaults to a non-deterministic algorithm. To explicitly enable determinism call torch.use_deterministic_algorithms(True, warn_only=False). (Triggered internally at /pytorch/aten/src/ATen/native/transformers/cuda/attention_backward.cu:110.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
quantized 7/7 layers in the block, loss iter 0: 0.000117 -> iter 738: 0.000050,'peak_ram': 33.24GB, 'peak_vram': 28.59GB
Quantizing done: 100%|██████████| 36/36 [2:10:40<00:00, 217.79s/it]                          
2026-01-30 19:13:47 INFO device.py L1430: 'peak_ram': 33.24GB, 'peak_vram': 28.59GB
2026-01-30 19:13:47 INFO base.py L1794: quantization t

(Qwen3VLForConditionalGeneration(
   (model): Qwen3VLModel(
     (visual): Qwen3VLVisionModel(
       (patch_embed): Qwen3VLVisionPatchEmbed(
         (proj): Conv3d(3, 1152, kernel_size=(2, 16, 16), stride=(2, 16, 16))
       )
       (pos_embed): Embedding(2304, 1152)
       (rotary_pos_emb): Qwen3VLVisionRotaryEmbedding()
       (blocks): ModuleList(
         (0-26): 27 x Qwen3VLVisionBlock(
           (norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
           (norm2): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
           (attn): Qwen3VLVisionAttention(
             (qkv): Linear(in_features=1152, out_features=3456, bias=True)
             (proj): Linear(in_features=1152, out_features=1152, bias=True)
           )
           (mlp): Qwen3VLVisionMLP(
             (linear_fc1): Linear(in_features=1152, out_features=4304, bias=True)
             (linear_fc2): Linear(in_features=4304, out_features=1152, bias=True)
             (act_fn): GELUTanh()
           

In [20]:
base_name = MODEL_ID.split("/")[-1]
    
if "AWQ" in TARGET_FORMATS:
        save_dir = os.path.join(OUTPUT_BASE_DIR, "W4A16-AWQ")
        print("\n💾 Exporting to AWQ...")
        ar.save_quantized(save_dir, format="auto_awq", inplace=False)
        push_to_hub(save_dir, f"{base_name}-W4A16-AutoRound-AWQ", HF_TOKEN)

if "GPTQ" in TARGET_FORMATS:
        save_dir = os.path.join(OUTPUT_BASE_DIR, "W4A16-GPTQ")
        print("\n💾 Exporting to GPTQ...")
        ar.save_quantized(save_dir, format="auto_gptq", inplace=False)
        push_to_hub(save_dir, f"{base_name}-W4A16-AutoRound-GPTQ", HF_TOKEN)

if "AutoRound" in TARGET_FORMATS:
        save_dir = os.path.join(OUTPUT_BASE_DIR, "W4A16-AutoRound")
        print("\n💾 Exporting to AutoRound format...")
        ar.save_quantized(save_dir, format="auto_round", inplace=False)
        push_to_hub(save_dir, f"{base_name}-W4A16-AutoRound", HF_TOKEN)



💾 Exporting to GPTQ...


packing model.language_model.layers.35.mlp.down_proj: 100%|██████████| 306/306 [00:07<00:00, 41.30it/s]   



[Hub] Pushing ./AutoRound/W4A16-GPTQ to Vishva007/Qwen3-VL-8B-Instruct-W4A16-AutoRound-GPTQ...


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

[Hub] ✅ Successfully uploaded: https://huggingface.co/Vishva007/Qwen3-VL-8B-Instruct-W4A16-AutoRound-GPTQ
